In [ ]:
# Load Data 
import h5py
import json
import numpy as np
from numpy import argwhere


# Open the dataset
hdf5_file = h5py.File("/content/drive/MyDrive/EGH400-2/Dataset/GOLD_XYZ_OSC.0001_1024.hdf5",  'r')
# Load the modulation classes. You can also copy and paste the content of classes-fixed.txt.
modulation_classes = json.load(open("/content/drive/MyDrive/EGH400-2/Dataset/classes-fixed.json", 'r'))


# Read the HDF5 groups
data = hdf5_file['X'] # Data of length with 2555904 shape 0f (2555904,1024,2) 106496 dataset for each modulation, Starting point of zero noise level: 41000 for each data
modulation_onehot = hdf5_file['Y'] # Data which include name of modulation methods, length with 2555904
snr = hdf5_file['Z'] # noise level of data ,length with 2555904 (Max:30 , Min:-20)

class_name = ['OOK', '4ASK', '8ASK', 'BPSK', 'QPSK', '8PSK', '16PSK', '32PSK', '16APSK', '32APSK', '64APSK', '128APSK', '16QAM', '32QAM', '64QAM', '128QAM', '256QAM', 'AM-SSB-WC', 'AM-SSB-SC', 'AM-DSB-WC', 'AM-DSB-SC', 'FM', 'GMSK', 'OQPSK']





In [ ]:
# Preprosessing data

n = 12500 # Numbers of samples for each modulation method


new_data = np.zeros(shape= (n*24,1024,2) ) # Create zeros array to store fileted data
new_modulation = np.zeros(shape = (n*24,24) ) # Create zeros array to store fileted data

  

# Store low noise level data to zeros array.
for i in range(0,24):
  new_data[i*n:i*n+n] = data[i*106496 + 41000 : i*106496+ 41000 + n ]
  new_modulation[i*n:i*n+n] = modulation_onehot[i*106496 + 41000:i*106496+ 41000 + n]


new_data = new_data*0.1 # Rescaling data -2 ~ 2 to -0.2 ~ 2 
  

In [ ]:
# Split the Data to Train & Test set

from sklearn.model_selection import train_test_split

# Convert data to numpy array to split dataset for training and testing (validation).
x = np.array(new_data)
y = np.array(new_modulation)

# Divide data to train and test. 15 percent of datasts is used to validation and test 
x_train, x_test, y_train, y_test = train_test_split(x,y, 
                                                      test_size = 0.15,
                                                      random_state = 1,
                                                      shuffle = True,
                                                      stratify = y)


In [ ]:
#Implement Model 

import tensorflow as tf
import numpy as np
import time  
from tensorflow.keras.callbacks import TensorBoard
from matplotlib import pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping

x_train = x_train.reshape(x_train.shape[0],512,2,2)
x_test = x_test.reshape(x_test.shape[0],512,2,2)


model = tf.keras.Sequential([
    
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal', input_shape = (512, 2, 2)), 
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1), 
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.1), 
    tf.keras.layers.Conv2D( 32, (2,2), padding = "same", activation = "relu", input_shape = (512,2,2)),    
    tf.keras.layers.MaxPooling2D( (2,2) ),  
    tf.keras.layers.BatchNormalization(axis = -1 ),  
    tf.keras.layers.Dropout(0.5),      
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation = 'relu'),         
    tf.keras.layers.Dense(1024, activation = 'relu'),           
    tf.keras.layers.Dense(1024, activation = 'relu'),           
    tf.keras.layers.Dense(1024, activation = 'relu'),          
    tf.keras.layers.Dense(1024, activation = 'relu'),    
    tf.keras.layers.Dense(1024, activation = 'relu'),
    tf.keras.layers.Dense(1024, activation = 'relu'),       
    tf.keras.layers.Dense(1024, activation = 'relu'),          
    tf.keras.layers.Dense(1024, activation = 'relu'),
    tf.keras.layers.Dense(1024, activation = 'relu'), 
    tf.keras.layers.Dense(1024, activation = 'relu'), 
    tf.keras.layers.Dense(1024, activation = 'relu'),                                 
    tf.keras.layers.Dense(24, activation = 'softmax') 
    
])

model.summary()

tensorboard = TensorBoard( log_dir = 'logs/{}'.format('Model' + str( int(time.time()) ) ))

es = EarlyStopping( monitor = 'val_loss', patience = 5)

model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ['accuracy'])
model.fit(x_train ,y_train,validation_data = (x_test,y_test) ,epochs = 300, callbacks = [ tensorboard , es ])

result = model.evaluate([x_test],[y_test]) 
print(result)




In [ ]:
%load_ext tensorboard 

In [ ]:
%tensorboard --logdir logs